# Chromatography

Let's simulate the chromatographic process!
To do this, we will use a very simple cell model.

## Cell Model

We imagine the stationary (i.e. the column) and the mobile phase to consist of cells.
Each cell of the stationary phase is in contact with one cell from the mobile phase.
We distribute the molecules between the stationary and mobile phase according to the equilibrium constant.
When this is done, we move the mobile phase cells 'down one cell' such that they are in contact with the next
cell of the stationary phase. Note: we use zero-based indexing for the cells (as does Python), so the first cell is cell 0.

![Cell Model](./img/cell-model.jpg)

## Equilibrium

The molecules will partition between their bound and free state according to an equilibrium constant $K$.
Since we will assume all the cells to be the same size (i.e. have the same volume) we can calculate with
amounts of molecules directly and do not need to bother with concentrations.

$$K = \frac{c_{bound}}{c_{free}} = \frac{n_{bound}/V_{cell}}{n_{free}/V_{cell}} = \frac{n_{bound}}{n_{free}}$$

Computationally, it is much easier to reformulate the equilibrium constant as a fraction of bound molecules give the total number of molecules.

$$f_{bound} = \frac{K}{1 + K} \cdot n_{total}$$

The number of free molecules is then simply $n_{free} = n_{total} - n_{bound}$.

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from math import floor
from ipywidgets import fixed

## Single Compound, Equilibrium

In [2]:
def simulate_column_1(n_cells: int, n_steps: int, n_molecules: int, K: float):
    # Reformat K to a fractional entity
    frac_bound = K / (1 + K)

    # Initial state (zero everywhere)
    stationary_phase = np.zeros(n_cells)
    mobile_phase = np.zeros(n_cells + n_steps)

    # Loading of mobile phase in cell just before the stationary phase
    mobile_phase[n_cells] = n_molecules

    for step in range(1, n_steps + 1):
        # Equilibration
        for cell in range(n_cells):
            n_tot = stationary_phase[cell] + mobile_phase[cell + step]
            n_bound = floor(n_tot * frac_bound)
            n_free = n_tot - n_bound
            stationary_phase[cell] = n_bound
            mobile_phase[cell + step] = n_free

    fig, ax = plt.subplots()
    ax.plot(mobile_phase)
    ax.set_xlim(0, n_steps)
    return fig

In [ ]:
n_cell_widget = widgets.IntSlider(
    value=100, min=50, max=500, step=50, description="Number of Cells"
)
k_widget = widgets.BoundedFloatText(
    value=3.0, min=0.1, max=10, description="Equilibrium constant K"
)

ui = widgets.HBox([n_cell_widget, k_widget])
out = widgets.interactive_output(
    simulate_column_1,
    dict(
        n_cells=n_cell_widget,
        n_steps=fixed(1000),
        n_molecules=fixed(int(1e10)),
        K=k_widget,
    ),
)

display(ui, out)

Output()